In [1]:
import os
import keras
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten, Dropout
from keras.layers.core import Dense
from keras.datasets import cifar10
from keras.optimizers import RMSprop
from keras.callbacks import TensorBoard, ModelCheckpoint

Using TensorFlow backend.


In [2]:
# data prepararion
class CIFAR10Dataset():
    def __init__(self):
        
        # image shape
        self.image_shape = (32,32,3)
        self.num_classes = 10
        
    def get_batch(self):
        (X_train, y_train), (X_test, y_test) = cifar10.load_data()
        X_train, X_test = [self.preprocess(d) for d in [X_train, X_test]]
        y_train, y_test = \
        [self.preprocess(d, label_data=True) for d in [y_train, y_test]]
        return X_train, y_train, X_test, y_test
      
        
    def preprocess(self, data, label_data=False):
        # for labels
        if label_data:
            # convert class vectors to binary class matrices
            data = keras.utils.to_categorical(data, self.num_classes)
            
        # for pixels data
        else:
            data = data.astype('float32')
            # convert the value to 0~1 scale (pixel's max value = 255)
            data = data / 255
            # add data set length to top
            shape = (data.shape[0], ) + self.image_shape
            data = data.reshape(shape)
            
        return data

In [3]:
# LeNet network
def network(input_shape, num_classes):
    model = Sequential()
    
    # extract image features by convolution and max pooling layers
    model.add(Conv2D(32, kernel_size=3, padding='same',
                    input_shape=input_shape, activation='relu'))
    
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))
    
    # classifiy the class by fully-connected layers
    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))
    return model

In [4]:
class Trainer():
    def __init__(self, model, loss, optimizer):
        self._target = model
        self._target.compile(loss=loss, optimizer=optimizer,metrics=['accuracy'])
        self.verbose = 1
        self.log_dir = os.path.join(os.path.dirname(os.path.abspath('__file__')),'logdir')
        self.model_file_name = 'model_file.hdf5'
        
    def train(self, X_train, y_train, batch_size, epochs, validation_split):
        if os.path.exists(self.log_dir):
            import shutil
            # remove previous execution
            shutil.rmtree(self.log_dir)
        else:
            pass
        os.mkdir(self.log_dir)
        
        self._target.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs, 
            validation_split=validation_split,
            callbacks=[TensorBoard(log_dir=self.log_dir),
                      ModelCheckpoint(os.path.join(self.log_dir,
                                    self.model_file_name),
                                      save_best_only=True)],
             verbose=self.verbose
        )

In [5]:
dataset = CIFAR10Dataset()

In [6]:
# make model
model = network(dataset.image_shape, dataset.num_classes)

W0829 05:50:11.420139 29128 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0829 05:50:11.453082 29128 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0829 05:50:11.459037 29128 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0829 05:50:11.496934 29128 deprecation_wrapper.py:119] From C:\Users\gudeg\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0829 05:50:11.502918 29128 deprecation_wrapper.py:119] From C:\Users\gudeg\Ana

In [11]:
# prepare data
X_train, y_train, X_test, y_test = dataset.get_batch()

# train the model
trainer = Trainer(model, loss='categorical_crossentropy',
                 optimizer=RMSprop())

trainer.train(X_train, y_train, batch_size=128, epochs=15, validation_split=0.2)

Train on 40000 samples, validate on 10000 samples
Epoch 1/15
40000/40000 [==============================] - 74s 2ms/step - loss: 0.9808 - acc: 0.6739 - val_loss: 0.9700 - val_acc: 0.6840
Epoch 2/15
40000/40000 [==============================] - 73s 2ms/step - loss: 0.9279 - acc: 0.6902 - val_loss: 0.8044 - val_acc: 0.7322
Epoch 3/15
40000/40000 [==============================] - 73s 2ms/step - loss: 0.9116 - acc: 0.6940 - val_loss: 0.9536 - val_acc: 0.7137
Epoch 4/15
40000/40000 [==============================] - 73s 2ms/step - loss: 0.9000 - acc: 0.6946 - val_loss: 0.8122 - val_acc: 0.7275
Epoch 5/15
40000/40000 [==============================] - 74s 2ms/step - loss: 0.9010 - acc: 0.6979 - val_loss: 0.8936 - val_acc: 0.7214
Epoch 6/15
40000/40000 [==============================] - 73s 2ms/step - loss: 0.8955 - acc: 0.6995 - val_loss: 0.8324 - val_acc: 0.7236
Epoch 7/15
40000/40000 [==============================] - 73s 2ms/step - loss: 0.8837 - acc: 0.7011 - val_loss: 0.8683 - val_acc

In [12]:
# show result
score = model.evaluate(X_test, y_test, verbose=0)

In [13]:
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.8480859862327575
Test accuracy: 0.7149
